In [75]:
import pandas as pd

In [76]:
task_csv = pd.read_csv('task.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])
train_csv = pd.read_csv('train.csv', delimiter=';', names=["id", "user_id", "movie_id", "rating"])
train_df = train_csv.pivot(index='movie_id',columns='user_id',values='rating')
train_df

user_id,5,12,19,24,31,52,62,63,68,69,70,71,78,80,90,92,93,105,106,109,118,125,131,139,140,148,149,151,154,156,165,169,175,176,180,181,183,188,189,196,...,1618,1628,1631,1641,1642,1648,1659,1662,1664,1665,1667,1676,1677,1686,1689,1691,1708,1709,1712,1714,1716,1718,1722,1724,1728,1732,1738,1750,1758,1759,1767,1769,1779,1781,1785,1796,1804,1813,1815,1816
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,5.0,NaN,NaN,3.0,1.0,NaN,5.0,1.0,2.0,3.0,3.0,NaN,NaN,NaN,3.0,4.0,3.0,3.0,5.0,5.0,3.0,4.0,5.0,3.0,5.0,NaN,4.0,5.0,0.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,3.0,5.0,NaN,5.0,NaN,4.0,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,4.0,5.0,2.0,NaN,5.0,4.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,3.0,NaN,2.0
2,2.0,NaN,2.0,5.0,0.0,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,4.0,3.0,4.0,1.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,4.0,4.0,2.0,NaN,NaN,...,4.0,NaN,NaN,3.0,3.0,5.0,NaN,NaN,4.0,NaN,4.0,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,5.0,NaN,1.0,3.0,5.0,NaN,NaN
3,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,3.0,NaN,3.0,NaN,NaN,3.0,2.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,5.0,2.0,NaN,2.0,5.0,4.0,2.0
4,NaN,5.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,3.0,NaN,NaN,NaN,NaN,2.0,3.0,4.0,NaN,5.0,1.0,NaN,NaN,5.0,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,...,NaN,NaN,5.0,3.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,3.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,5.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,3.0,4.0,NaN,3.0,4.0
5,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,5.0,4.0,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,3.0,3.0,4.0,3.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,5.0,0.0,NaN,5.0,NaN,3.0,4.0,4.0,5.0,3.0,NaN,NaN,3.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,4.0,2.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,5.0,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,3.0,NaN,4.0,NaN,...,NaN,NaN,2.0,NaN,5.0,NaN,3.0,1.0,NaN,NaN,2.0,NaN,4.0,2.0,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,4.0
197,4.0,NaN,5.0,3.0,NaN,3.0,3.0,5.0,NaN,0.0,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,5.0,1.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,2.0,5.0,...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,NaN,2.0,5.0,4.0,3.0,3.0,NaN,5.0,3.0,NaN,3.0,NaN,NaN,4.0,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0
198,NaN,NaN,NaN,2.0,4.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,3.0,5.0,2.0,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,2.0,3.0,NaN,...,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,5.0,NaN,NaN,5.0,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,2.0


In [77]:
def get_similar_users(user_id):
  users = []
  for usr_id, ratings in train_df.iteritems():
    if usr_id != user_id:
      users.append((usr_id, sum(abs(train_df[user_id] - ratings).fillna(0))))
  
  return sorted(users, key=lambda x: x[1])
temp_users = get_similar_users(1642)

In [78]:
def average(lst):
    return sum(lst) / len(lst)

In [79]:
def predict_user_rating(movie_id, similar_users, user_id):
  best_ratings = []
  for similar_user in similar_users:
    movie_rating = train_csv[(train_csv['movie_id'] == movie_id) & (train_csv['user_id'] == similar_user[0])]['rating']
    if len(movie_rating.values) > 0:
      best_ratings.append(movie_rating.values[0])
    if len(best_ratings) >= 3:
      return str(int(round(average(best_ratings))))
  return str(int(round(average(best_ratings))))

predict_user_rating(137, temp_users, 1642)

'4'

In [ ]:
temp_user_id = -1
for index, row in task_csv.iterrows():
  if temp_user_id != int(row['user_id']):
    similar_users = get_similar_users(int(row['user_id']))
  task_csv.at[index, 'rating'] = predict_user_rating(int(row['movie_id']), similar_users, int(row['user_id']))
  temp_user_id = int(row['user_id'])
  print(index, row['user_id'])
task_csv

In [81]:
task_csv.to_csv('submission.csv', sep=';', encoding='utf-8', index=False, header=False)